In [1]:
#importing necesaary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf
import keras
%matplotlib inline

In [2]:
#loading our datasets
credits = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Movie Recommendor System/tmdb_5000_credits.csv')
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Movie Recommendor System/tmdb_5000_movies.csv')

In [3]:
#data shape
movies.head(1)
movies.shape

(4803, 20)

In [4]:
credits.head(1)
credits.shape

(4803, 4)

In [5]:
movies.shape

(4803, 20)

In [6]:
#merging the features of movies and credits all together
movies = movies.merge(credits, on='title')

In [7]:
movies.shape

(4809, 23)

In [8]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [9]:
movies.head(1)
movies['vote_average']
movies.dtypes

budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
movie_id                  int64
cast                     object
crew                     object
dtype: object

In [10]:
# drop the non useful columns and create a separate dataset for our model
# important features include genre , rating , actor cast ,id  etc.
movies_train = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [11]:
movies_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [12]:
#to check if some of the datasets are empty
movies_train.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [13]:
movies_train.dropna(inplace = True)

<ipython-input-13-b83518668558>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train.dropna(inplace = True)


In [14]:
movies_train.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [15]:
#to check for duplicate entities
movies_train.duplicated().sum()

0

In [16]:
movies_train.drop_duplicates(inplace = True)

<ipython-input-16-b1e9c4c7631d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train.drop_duplicates(inplace = True)


In [17]:
movies_train.duplicated().sum()

0

In [18]:
#getting genres of first entry
movies_train.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [19]:
#ast.literal_eval is used to convert the dictionary of strings to a list with dictionaries
import ast
def change(obj):
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [20]:
movies_train['genres'] = movies_train['genres'].apply(change)

<ipython-input-20-56917faa5c9c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train['genres'] = movies_train['genres'].apply(change)


In [21]:
movies_train.iloc[0].genres

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [22]:
movies_train.iloc[0].keywords

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [23]:
movies_train['keywords'] = movies_train['keywords'].apply(change)

<ipython-input-23-1f2495704de1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train['keywords'] = movies_train['keywords'].apply(change)


In [24]:
movies_train.iloc[0].keywords

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [25]:
def change1(obj):
  L = []
  iter = 0
  for i in ast.literal_eval(obj):
    if(iter!=5):
      L.append(i['name'])
      iter+=1
    else:
      break
  return L

In [26]:
movies_train['cast'] = movies_train['cast'].apply(change1)

<ipython-input-26-2b6845bc2f5e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train['cast'] = movies_train['cast'].apply(change1)


In [27]:
movies_train.iloc[0].cast

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez']

In [28]:
def get_director(obj):
  L = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      L.append(i['name'])
      break
  return L

In [29]:
movies_train['crew'] = movies_train['crew'].apply(get_director)

<ipython-input-29-6fb8254f2ae3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train['crew'] = movies_train['crew'].apply(get_director)


In [30]:
movies_train.iloc[0].crew

['James Cameron']

In [31]:
movies_train.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]


In [32]:
movies_train['overview'] = movies_train['overview'].apply(lambda x:x.split())

<ipython-input-32-406719aa5436>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train['overview'] = movies_train['overview'].apply(lambda x:x.split())


In [33]:
movies_train['genres']= movies_train['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies_train['keywords']= movies_train['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies_train['cast']= movies_train['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies_train['crew']= movies_train['crew'].apply(lambda x:[i.replace(" ","") for i in x])

<ipython-input-33-79e639712ad7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train['genres']= movies_train['genres'].apply(lambda x:[i.replace(" ","") for i in x])
<ipython-input-33-79e639712ad7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train['keywords']= movies_train['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
<ipython-input-33-79e639712ad7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [34]:
movies_train.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]


In [35]:
movies_train ['tags'] = movies_train['overview'] + movies_train['genres'] + movies_train['keywords'] + movies_train['cast'] + movies_train['crew']

<ipython-input-35-386a09084e64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train ['tags'] = movies_train['overview'] + movies_train['genres'] + movies_train['keywords'] + movies_train['cast'] + movies_train['crew']


In [36]:
movies_train.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [37]:
movies_train_new = movies_train[['movie_id', 'title', 'tags']]

In [38]:
movies_train_new.head(1)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [39]:
movies_train_new.head(1)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [40]:
movies_train_new['tags'] = movies_train_new['tags'].apply(lambda x: " ".join(x))

<ipython-input-40-f7b0211bcfe0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train_new['tags'] = movies_train_new['tags'].apply(lambda x: " ".join(x))


In [41]:
movies_train_new.head(1)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [42]:
movies_train_new['tags'] = movies_train_new['tags'].apply(lambda x:x.lower())

<ipython-input-42-e6c0dfc2da40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train_new['tags'] = movies_train_new['tags'].apply(lambda x:x.lower())


In [43]:
movies_train_new.head(1)


,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."


In [44]:
import nltk

In [45]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [46]:
#this funtion is used to covert the words to it roots
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [47]:
movies_train_new['tags'] = movies_train_new['tags'].apply(stem)

<ipython-input-47-f201e9b56462>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_train_new['tags'] = movies_train_new['tags'].apply(stem)


In [48]:
#converting the texts into vector on the basis of frequency
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [49]:
vectors = cv.fit_transform(movies_train_new['tags']).toarray()

In [50]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [51]:
from sklearn.metrics.pairwise import cosine_similarity

In [52]:
similarity = cosine_similarity(vectors)

In [53]:
similarity

array([[1.        , 0.08006408, 0.08492078, ..., 0.04441156, 0.        ,
        0.        ],
       [0.08006408, 1.        , 0.05892557, ..., 0.02311251, 0.        ,
        0.02541643],
       [0.08492078, 0.05892557, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04441156, 0.02311251, 0.02451452, ..., 1.        , 0.0418121 ,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.0418121 , 1.        ,
        0.0919601 ],
       [0.        , 0.02541643, 0.        , ..., 0.04229549, 0.0919601 ,
        1.        ]])

In [83]:
#importing libraries for displaying the output
from ipywidgets import widgets
from IPython.display import display ,clear_output , HTML

In [84]:
#this creates a dropdown of list of movies to select
dropdown = widgets.Dropdown(
    options=movies['title'],
    description='Movies:',
    disabled=False,
)

In [85]:
#this function creates a list of similar movies
#filter top 5 movies
#then generates search links for the same
def recommend(movie):
    if movie not in movies_train_new['title'].values:
        return []

    movie_index = movies_train_new[movies_train_new['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    recommended_movies = []
    for i in movies_list:
        movie_title = movies_train_new.iloc[i[0]].title
        search_url = f"https://www.google.com/search?q={movie_title.replace(' ', '+')}+movie"
        recommended_movies.append(f'<a href="{search_url}" target="_blank">{movie_title}</a>')
    return recommended_movies


In [87]:
#on executing this part this generates the drop down after which one can search for it
display(dropdown)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_movie = change['new']
        recommendations = recommend(selected_movie)
        clear_output(wait=True)
        display(dropdown)
        print(f"Recommendations for {selected_movie}:")
        for rec in recommendations:
            display(HTML(rec))

dropdown.observe(on_change, names='value')

Dropdown(description='Movies:', index=938, options=('Avatar', "Pirates of the Caribbean: At World's End", 'Spe…